In [ ]:
# Removes lint errors from VS Code
from typing import Dict, TYPE_CHECKING, Tuple, List

if TYPE_CHECKING:
    import kedro

    catalog: kedro.io.data_catalog.DataCatalog
    session: kedro.framework.session.session.KedroSession
    catalog: kedro.io.data_catalog.DataCatalog
    pipelines: Dict[str, kedro.pipeline.pipeline.Pipeline]

import numpy as np
import pandas as pd

patients_wrk: pd.DataFrame = catalog.load("mimic_mm_core.wrk.patients")
admissions_wrk: pd.DataFrame = catalog.load("mimic_mm_core.wrk.admissions")
transfers_wrk: pd.DataFrame = catalog.load("mimic_mm_core.wrk.transfers")

patients_ref: pd.DataFrame = catalog.load("mimic_mm_core.ref.patients")
admissions_ref: pd.DataFrame = catalog.load("mimic_mm_core.ref.admissions")
transfers_ref: pd.DataFrame = catalog.load("mimic_mm_core.ref.transfers")


2000-01-01 00:00:00,000 - kedro.io.data_catalog - INFO - Loading data from `mimic_mm_core.wrk.patients` (ParquetDataSet)...
2000-01-01 00:00:00,000 - kedro.io.data_catalog - INFO - Loading data from `mimic_mm_core.wrk.admissions` (ParquetDataSet)...
2000-01-01 00:00:00,000 - kedro.io.data_catalog - INFO - Loading data from `mimic_mm_core.wrk.transfers` (ParquetDataSet)...
2000-01-01 00:00:00,000 - kedro.io.data_catalog - INFO - Loading data from `mimic_mm_core.ref.patients` (ParquetDataSet)...
2000-01-01 00:00:00,000 - kedro.io.data_catalog - INFO - Loading data from `mimic_mm_core.ref.admissions` (ParquetDataSet)...
2000-01-01 00:00:00,000 - kedro.io.data_catalog - INFO - Loading data from `mimic_mm_core.ref.transfers` (ParquetDataSet)...


In [ ]:
tables = {
    "patients": [patients_wrk, patients_ref],
    "admissions": [admissions_wrk, admissions_ref],
    "transfers": [transfers_wrk, transfers_ref],
}


def is_categorical(table, col):
    # return metadata["tables"][table]["fields"][col]["type"] == "categorical"
    return col in [
        # Patient
        "gender",
        "anchor_year_group",
        "year_group",
        # Admissions
        "admission_type",
        "admission_location",
        "discharge_location",
        "insurance",
        "language",
        "marital_status",
        "ethnicity",
        "hospital_expire_flag",
        # Transfers
        "eventtype",
        "careunit",
    ]


def is_id(table, col):
    return col in ["subject_id", "hadm_id", "transfer_id"]


from pandas.api.types import (
    is_datetime64_any_dtype as is_datetime,
    is_timedelta64_dtype as is_timedelta,
)


In [ ]:
def gen_freq(a, b, cols, is_cat=None, fillna=1e-6, bins=32):
    if isinstance(cols, str):
        cols = [cols]
    if isinstance(is_cat, bool):
        is_cat = [is_cat]

    # Shallow copy a, b and only keep cols
    # FIXME: stop dropping NAs
    a, b = a[cols].dropna(), b[cols].dropna()

    # Convert any continuous variables to discrete
    if is_cat is not None and not all(is_cat):
        for col, cat in zip(cols, is_cat):
            if cat:
                continue

            if is_datetime(a[col]):
                c, d = a[col].dt.day_of_year, b[col].dt.day_of_year
            else:
                c, d = pd.to_numeric(a[col]), pd.to_numeric(b[col])

            col_bins = np.histogram_bin_edges(c, bins=bins).astype(np.float32)
            a[col] = np.digitize(c, col_bins)
            b[col] = np.digitize(d, col_bins)

    ## Calculate frequencies of discrete variables
    a, b = a.value_counts(), b.value_counts()
    c = pd.concat([a, b], axis=1)
    c = c / c.sum()
    c = c.fillna(value=fillna)
    c = c / c.sum()
    return c.iloc[:, 0], c.iloc[:, 1]


In [ ]:
from scipy.stats import chisquare

res = []
for name, (a, b) in tables.items():
    for col in a.keys():
        if is_categorical(name, col):
            k, j = gen_freq(a, b, col)
            chi, p = chisquare(k, j)
            # print(f"{name:12}.{col:20}: X^2={chi:3.3f} p={100*p:7.3f}%")
            res.append([name, col, chi, p])

res = pd.DataFrame(res, columns=["table", "col", "X^2", "p"])
res.set_index(keys=["table", "col"]).style.background_gradient(axis=0)


In [ ]:
from scipy.stats import ks_2samp

res = []
for name, (a, b) in tables.items():
    for col in a.keys():
        if is_categorical(name, col) or is_id(name, col):
            continue
        c, d = a[col].dropna(), b[col].dropna()
        ks, p = ks_2samp(c, d)
        res.append([name, col, ks, p])

res = pd.DataFrame(res, columns=["table", "col", "K-S", "p"])
res.set_index(keys=["table", "col"]).style.background_gradient(axis=0)


In [ ]:
def get_parent(table):
    match table:
        case "patients":
            return None
        case "admissions":
            return "patients"
        case "transfers":
            return "admissions"
        case _:
            assert False, "Table not found"


In [ ]:
from scipy.special import rel_entr

res = []
for name, (a, b) in tables.items():
    cols = sorted(a.keys())
    for i, col_i in enumerate(cols):
        if is_id(name, col_i):
            continue
        cat_i = is_categorical(name, col_i)

        for col_j in cols[i + 1 :]:
            if is_id(name, col_j):
                continue
            cat_j = is_categorical(name, col_j)

            k, j = gen_freq(a, b, [col_i, col_j], [cat_i, cat_j])
            kl = rel_entr(k, j).sum()
            kl_norm = 1 / (1 + kl)
            res.append([name, col_i, name, col_j, kl, kl_norm, len(k), cat_i, cat_j])

for table in tables:
    a, b = tables[table]
    cols_i = sorted(a.keys())
    parent = table

    while get_parent(parent):
        parent = get_parent(parent)

        c, d = tables[parent]
        a, b = a.join(c, rsuffix=parent), b.join(d, rsuffix=parent)
        cols_j = sorted(c.keys())

        for col_i in cols_i:
            if is_id(table, col_i):
                continue
            cat_i = is_categorical(table, col_i)

            for col_j in cols_j:
                if is_id(parent, col_j):
                    continue
                cat_j = is_categorical(parent, col_j)

                k, j = gen_freq(a, b, [col_i, col_j], [cat_i, cat_j])
                kl = rel_entr(k, j).sum()
                kl_norm = 1 / (1 + kl)
                res.append(
                    [table, col_i, parent, col_j, kl, kl_norm, len(k), cat_i, cat_j]
                )

res = pd.DataFrame(
    res,
    columns=[
        "table_i",
        "col_i",
        "table_j",
        "col_j",
        "kl",
        "kl_norm",
        "mlen",
        "cat_i",
        "cat_j",
    ],
)


In [ ]:
res


,table_i,col_i,table_j,col_j,kl,kl_norm,mlen,cat_i,cat_j
0,patients,birth_year,patients,dod,0.010786,0.989329,33,False,False
1,patients,birth_year,patients,gender,0.000007,0.999993,2,False,True
2,patients,birth_year,patients,year_group,0.000023,0.999977,4,False,True
3,patients,dod,patients,gender,0.026839,0.973863,66,False,True
4,patients,dod,patients,year_group,0.036170,0.965092,131,False,True
...,...,...,...,...,...,...,...,...,...
205,transfers,intime,patients,year_group,0.001184,0.998817,132,False,True
206,transfers,outtime,patients,birth_year,0.000571,0.999430,33,False,False
207,transfers,outtime,patients,dod,0.684176,0.593762,1026,False,False
208,transfers,outtime,patients,gender,0.000747,0.999254,66,False,True


In [ ]:
def mk_ks_plot(filter=None, val="kl_norm"):
    # idx_tuples = []
    # for table in reversed(tables):
    #     a, _ = tables[table]
    #     for col in sorted(a.keys()):
    #         if is_categorical(table, col):
    #             idx_tuples.append((table, col))

    # idx = pd.MultiIndex.from_tuples(idx_tuples, names=['table_i', 'col_i'])
    # cols = pd.MultiIndex.from_tuples(reversed(idx_tuples), names=['table_j', 'col_j'])

    pt = res[filter] if filter is not None else res
    pt = pt.pivot_table(
        values=val, index=["table_i", "col_i"], columns=["table_j", "col_j"]
    )

    idx_tuples = []
    for table in reversed(tables):
        idx_tuples.extend([i for i in pt.index if i[0] == table])
    idx = pd.MultiIndex.from_tuples(idx_tuples, names=["table_i", "col_i"])

    col_tuples = []
    for table in tables:
        col_tuples.extend([i for i in reversed(pt.columns) if i[0] == table])
    cols = pd.MultiIndex.from_tuples(col_tuples, names=["table_j", "col_j"])

    # Try to tweak colormap
    vmin = vmax = None
    match val:
        case "kl_norm":
            vmin = 0.996
        case "kl":
            vmax = 0.04
        case "mlen":
            vmax = 100

    pt = pt.reindex(index=idx, columns=cols)
    pt = pt.style.background_gradient(axis=None, vmin=vmin, vmax=vmax).applymap(
        lambda x: "color: transparent; background-color: transparent"
        if pd.isnull(x)
        else ""
    )

    return pt


In [ ]:
mk_ks_plot(res["cat_i"] & res["cat_j"])


In [ ]:
mk_ks_plot(res["cat_i"] & res["cat_j"], "mlen")


In [ ]:
mk_ks_plot(~res["cat_i"] & ~res["cat_j"])


In [ ]:
mk_ks_plot(res["cat_i"] ^ res["cat_j"])


In [ ]:
mk_ks_plot()


In [ ]:
mk_ks_plot(val="kl")
